In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np 
from sklearn.preprocessing import MultiLabelBinarizer
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.stats.diagnostic import het_white
import scipy
from statsmodels.compat import lzip

pd.set_option('display.max_columns', 500)
plt.rcParams["figure.figsize"] = (30, 10)

In [2]:
data = pd.read_csv('df.csv', low_memory=False)
data

,aso,brand,capacity,color,condition,features,first_owner,fuel_type,horse_power,mileage,model,no_accidents,number_of_doors,origin_country,price,price_currency,transmission,type,url,version,year
0,Tak,Rolls-Royce,6 749 cm3,Czarny,Używane,"ABS,Elektryczne szyby przednie,Poduszka powiet...",Tak,Benzyna,599 KM,49 000 km,Cullinan,Tak,5,NaN,1616999,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-cull...,Black Badge,2020
1,NaN,Rolls-Royce,6 750 cm3,Niebieski,Używane,NaN,NaN,Benzyna,335 KM,48 000 km,Silver Spur,NaN,5,NaN,298800,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-silv...,NaN,1991
2,Tak,Rolls-Royce,6 592 cm3,Szary,Używane,"ABS,Elektryczne szyby przednie,Poduszka powiet...",NaN,Benzyna,632 KM,1 800 km,Wraith,Tak,2,Włochy,1950000,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-wrai...,NaN,2019
3,NaN,Rolls-Royce,5 379 cm3,Biały,Używane,"ABS,Immobilizer,Alufelgi,Gniazdo SD,Podgrzewan...",Tak,Benzyna,326 KM,86 400 km,Silver Seraph,NaN,4,NaN,197000,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-silv...,NaN,1999
4,NaN,Rolls-Royce,4 000 cm3,Czarny,Używane,NaN,Tak,Benzyna,130 KM,50 000 km,Wraith,NaN,4,Wielka Brytania,25000,PLN,Manualna,NaN,https://www.otomoto.pl/oferta/rolls-royce-wrai...,NaN,1952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://carsmile.pl/oferta/peugeot/208/hatchba...,NaN,NaN
104434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://carsmile.pl/oferta/nissan/qashqai/cros...,NaN,NaN
104435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://carsmile.pl/oferta/nissan/qashqai/cros...,NaN,NaN
104436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://carsmile.pl/oferta/audi/a6/kombi-leasi...,NaN,NaN


In [3]:
data['aso'].value_counts()

Tak    43759
aso       32
Name: aso, dtype: int64

In [4]:
data = data.dropna(subset=['features', 'capacity', 'horse_power', 'mileage'])
data

,aso,brand,capacity,color,condition,features,first_owner,fuel_type,horse_power,mileage,model,no_accidents,number_of_doors,origin_country,price,price_currency,transmission,type,url,version,year
0,Tak,Rolls-Royce,6 749 cm3,Czarny,Używane,"ABS,Elektryczne szyby przednie,Poduszka powiet...",Tak,Benzyna,599 KM,49 000 km,Cullinan,Tak,5,NaN,1616999,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-cull...,Black Badge,2020
2,Tak,Rolls-Royce,6 592 cm3,Szary,Używane,"ABS,Elektryczne szyby przednie,Poduszka powiet...",NaN,Benzyna,632 KM,1 800 km,Wraith,Tak,2,Włochy,1950000,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-wrai...,NaN,2019
3,NaN,Rolls-Royce,5 379 cm3,Biały,Używane,"ABS,Immobilizer,Alufelgi,Gniazdo SD,Podgrzewan...",Tak,Benzyna,326 KM,86 400 km,Silver Seraph,NaN,4,NaN,197000,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-silv...,NaN,1999
5,NaN,Rolls-Royce,6 750 cm3,Biały,Używane,"Elektryczne szyby przednie,Klimatyzacja manual...",NaN,Benzyna,200 KM,650 000 km,Silver Shadow,NaN,4,NaN,59900,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-silv...,NaN,1975
6,Tak,Rolls-Royce,5 379 cm3,Zielony,Używane,"ABS,Elektryczne szyby przednie,Poduszka powiet...",NaN,Benzyna,326 KM,58 650 km,Silver Seraph,NaN,4,Kanada,189900,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-silv...,NaN,1999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104397,NaN,BMW,1 995 cm3,Czarny,Używane,"ABS,Elektrycznie ustawiane lusterka,Poduszka p...",NaN,Diesel,190 KM,6 000 km,Seria 4,NaN,5,NaN,261900,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/bmw-seria-4-420d...,420d Gran Coupe xDrive,2021
104398,NaN,BMW,1 995 cm3,Srebrny,Używane,"ABS,Elektryczne szyby przednie,Poduszka powiet...",NaN,Benzyna,150 KM,203 000 km,Seria 3,NaN,4,NaN,21800,PLN,Manualna,NaN,https://www.otomoto.pl/oferta/bmw-seria-3-2-0-...,320i,2007
104399,Tak,BMW,1 997 cm3,Niebieski,Używane,"ABS,Elektryczne szyby przednie,Poduszka powiet...",Tak,Benzyna,183 KM,145 000 km,Seria 4,Tak,2,Polska,65190,PLN,Manualna,NaN,https://www.otomoto.pl/oferta/bmw-seria-4-bmw-...,NaN,2014
104400,Tak,BMW,2 998 cm3,Czarny,Używane,"ABS,Elektrycznie ustawiane lusterka,Poduszka p...",Tak,Benzyna,340 KM,36 000 km,Seria 8,Tak,4,NaN,399750,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/bmw-seria-8-bmw-...,840i,2020


In [5]:
data.isna().sum()

aso                38325
brand                  0
capacity               0
color                  0
condition              0
features               0
first_owner        50273
fuel_type              0
horse_power            0
mileage                0
model                  0
no_accidents       29623
number_of_doors      343
origin_country     29145
price                  0
price_currency         0
transmission          24
type               80176
url                    0
version            31882
year                   0
dtype: int64

# Co do zrobienia z danymi? 
- aso na 0-1 X 
- OH brand X
- capacity na int X
- OH color X
- condition 0-1 X
- OH features X 
- first_owner bool X 
- fuel_type OH X 
- horse_power na int X 
- mileage  git na int X
- no_accidents X
- number_of_doors X 
- origin country OH 
- price - Y 
- transmission - if automatic 
- year 

In [6]:
def map_aso(row): 
    if row == 'Tak': 
        return 1 
    else: 
        return 0 

def map_capacity(row): 
    split_list = row.split(' ')
    try:  
        if len(split_list) == 2:
            return int(''.join(split_list[:1]))
        else: 
            return int(''.join(split_list[:2]))
            
    except ValueError: 
        return row
        
def map_condition(row): 
    if row == 'Używane':
        return 0
    else: 
        return 1 

def map_transmission(row): 
    if row == 'Automatyczna': 
        return 1 
    else :
        return 0 

In [7]:
data = data.dropna(subset=['features', 'capacity', 'horse_power', 'mileage'])

lb = MultiLabelBinarizer()
features = lb.fit_transform(data['features'])
features = pd.DataFrame(features, columns=lb.classes_)

df = data.loc[data['fuel_type'].isin(['Benzyna', 'Diesel', 'Hybryda', 'Benzyna+LPG', 'Elektryczny'])]

print(data.shape)
df = pd.concat([df.drop(columns='brand'), pd.get_dummies(df['brand'], drop_first=True)], axis=1)
df = pd.concat([df.drop(columns='fuel_type'), pd.get_dummies(df['fuel_type'], drop_first=True)], axis=1)


df['aso'] = df['aso'].dropna().apply(map_aso)
df['capacity'] = df['capacity'].dropna().apply(map_capacity)
df['horse_power'] = df['horse_power'].dropna().apply(map_capacity)
df['mileage'] = df['mileage'].dropna().apply(map_capacity)
df['no_accidents'] = df['no_accidents'].apply(map_aso)
df['first_owner'] = df['first_owner'].apply(map_aso)
df['condition'] = df['condition'].apply(map_condition)

df['transmission'] = df['transmission'].apply(map_transmission)
df = (df.rename(columns={'transmission': 'automatyczna', 'condition': 'new'})
        .fillna(0)
        .drop(columns=['origin_country', 'model', 'type', 'url', 'version', 'price_currency', 'color', 'features'])
        .dropna()
        .astype(int)
        .reset_index(drop=True))

print(df.shape)


(80208, 21)
(80165, 41)


In [8]:
Y = df['price']
X = df.loc[:, df.columns != 'price']
X = sm.add_constant(X)
lm = sm.OLS(Y, X).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.746
Model:                            OLS   Adj. R-squared:                  0.746
Method:                 Least Squares   F-statistic:                     5886.
Date:                Mon, 17 Jan 2022   Prob (F-statistic):               0.00
Time:                        20:53:56   Log-Likelihood:            -1.0034e+06
No. Observations:               80165   AIC:                         2.007e+06
Df Residuals:                   80124   BIC:                         2.007e+06
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            -3.97e+06   1.41e+05    -

In [9]:
# X2 = df.drop(columns=lb.classes_).drop(columns='price')
# Y = df['price']
# lm2 = sm.OLS(Y, X2).fit()
# print(lm2.summary())

In [10]:
def get_model(Y, X): 
    lm =  sm.OLS(Y, X).fit()
    print(lm.summary())
    return lm

In [63]:
def print_all_tests(lm):
    name = ["Lagrange multiplier statistic", "p-value", "f-value", "f p-value"]
    test = sms.het_breuschpagan(lm.resid, lm.model.exog)
    print('breuschpagan')
    print(lzip(name, test))

    name = ["F statistic", "p-value"]
    test = sms.het_goldfeldquandt(lm.resid, lm.model.exog)
    print('goldfeldquandt')
    print(lzip(name, test))

    name = ["Jarque-Bera", "Chi^2 two-tail prob.", "Skew", "Kurtosis"]
    test = sms.jarque_bera(lm.resid)
    print('Jarque-Bera')
    # duza proba wiec olewamy 
    # jezeli mala proba i nie przechdozi to nie mozemy uzywac p-value 
    print(lzip(name, test))

    print(np.linalg.cond(lm.model.exog))

    print(sms.linear_reset(lm))
    # nie wierzymy w oszacowania parametrow 
    
    # jak mamy autokorelacja to nie wierzymy w std err i p value 
    # tak samo jak jest heteroskedastycznosc 



    # labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']
    # white_test = sms.het_white(lm.resid,  lm.model.exog)
    # print(dict(zip(labels, white_test)))



In [39]:
name = ["Lagrange multiplier statistic", "p-value", "f-value", "f p-value"]
test = sms.het_breuschpagan(lm.resid, lm.model.exog)
lzip(name, test)

[('Lagrange multiplier statistic', 19139.357516741704),
 ('p-value', 0.0),
 ('f-value', 628.2284869397796),
 ('f p-value', 0.0)]

In [40]:
name = ["F statistic", "p-value"]
test = sms.het_goldfeldquandt(lm.resid, lm.model.exog)
lzip(name, test)

[('F statistic', 0.9198685405565407), ('p-value', 0.9999999999999999)]

In [41]:
name = ["Jarque-Bera", "Chi^2 two-tail prob.", "Skew", "Kurtosis"]
test = sms.jarque_bera(lm.resid)
lzip(name, test)

[('Jarque-Bera', 77643218.63158967),
 ('Chi^2 two-tail prob.', 0.0),
 ('Skew', 4.652571647749513),
 ('Kurtosis', 155.17896384962773)]

In [42]:
np.linalg.cond(lm.model.exog)

93354762.56098351

In [43]:
print(sms.linear_reset(lm))

/home/kuba1302/UW/ekonometria_projekt/venv/lib/python3.9/site-packages/statsmodels/stats/diagnostic.py:1081: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  aug = res.fittedvalues[:, None]


<Wald test (chi2): statistic=86219.0548228618, p-value=0.0, df_denom=2>


In [44]:
print_all_tests(lm)

breuschpagan
[('Lagrange multiplier statistic', 19139.357516741704), ('p-value', 0.0), ('f-value', 628.2284869397796), ('f p-value', 0.0)]
goldfeldquandt
[('F statistic', 0.9198685405565407), ('p-value', 0.9999999999999999)]
Jarque-Bera
[('Jarque-Bera', 77643218.63158967), ('Chi^2 two-tail prob.', 0.0), ('Skew', 4.652571647749513), ('Kurtosis', 155.17896384962773)]
93354762.56098351
<Wald test (chi2): statistic=86219.0548228618, p-value=0.0, df_denom=2>


In [45]:
lm3 = get_model(Y=df['price'].apply(np.log), X=X)
print_all_tests(lm3)

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.879
Model:                            OLS   Adj. R-squared:                  0.879
Method:                 Least Squares   F-statistic:                 1.462e+04
Date:                Mon, 17 Jan 2022   Prob (F-statistic):               0.00
Time:                        21:16:37   Log-Likelihood:                -34355.
No. Observations:               80165   AIC:                         6.879e+04
Df Residuals:                   80124   BIC:                         6.917e+04
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            -146.1070      0.794   -1

/home/kuba1302/UW/ekonometria_projekt/venv/lib/python3.9/site-packages/statsmodels/stats/diagnostic.py:1081: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  aug = res.fittedvalues[:, None]


<Wald test (chi2): statistic=5885.175236731658, p-value=0.0, df_denom=2>


In [46]:
X.assign(mileage=lambda x: x['mileage'].apply(np.log))

,const,aso,capacity,new,first_owner,horse_power,mileage,no_accidents,number_of_doors,automatyczna,year,Audi,BMW,Bentley,Chevrolet,Citroën,Dacia,Ferrari,Fiat,Kia,Lamborghini,Land Rover,Lexus,MINI,Maserati,Mazda,McLaren,Mercedes-Benz,Mitsubishi,Porsche,Renault,Rolls-Royce,Saab,Seat,Suzuki,Toyota,Volvo,Škoda,Benzyna+LPG,Diesel,Hybryda
0,1.0,1,6749,0,1,599,10.799576,1,5,1,2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,1.0,1,6592,0,0,632,7.495542,1,2,1,2019,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,1.0,0,5379,0,1,326,11.366743,0,4,1,1999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,1.0,0,6750,0,0,200,13.384728,0,4,1,1975,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,1.0,1,5379,0,0,326,10.979343,0,4,1,1999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80160,1.0,0,1995,0,0,190,8.699515,0,5,1,2021,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
80161,1.0,0,1995,0,0,150,12.220961,0,4,0,2007,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
80162,1.0,1,1997,0,1,183,11.884489,1,2,0,2014,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
80163,1.0,1,2998,0,1,340,10.491274,1,4,1,2020,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
lm4 = get_model(Y=df['price'].apply(np.log), X=X.assign(mileage=lambda x: x['mileage'].apply(np.log)))
print_all_tests(lm4)

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.867
Model:                            OLS   Adj. R-squared:                  0.867
Method:                 Least Squares   F-statistic:                 1.303e+04
Date:                Mon, 17 Jan 2022   Prob (F-statistic):               0.00
Time:                        21:20:54   Log-Likelihood:                -38377.
No. Observations:               80165   AIC:                         7.684e+04
Df Residuals:                   80124   BIC:                         7.722e+04
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            -176.6092      0.761   -2

/home/kuba1302/UW/ekonometria_projekt/venv/lib/python3.9/site-packages/statsmodels/stats/diagnostic.py:1081: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  aug = res.fittedvalues[:, None]


<Wald test (chi2): statistic=10604.90779568241, p-value=0.0, df_denom=2>


In [62]:
X3 = X.assign(mileage=lambda x: x['mileage'].apply(np.log), year=lambda x: x['year'].apply(np.log))
lm4 = get_model(Y=df['price'].apply(np.log), X=X3.dropna())
print_all_tests(lm4)

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.866
Model:                            OLS   Adj. R-squared:                  0.866
Method:                 Least Squares   F-statistic:                 1.298e+04
Date:                Mon, 17 Jan 2022   Prob (F-statistic):               0.00
Time:                        21:25:45   Log-Likelihood:                -38513.
No. Observations:               80165   AIC:                         7.711e+04
Df Residuals:                   80124   BIC:                         7.749e+04
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const           -1407.5896      5.749   -2

/home/kuba1302/UW/ekonometria_projekt/venv/lib/python3.9/site-packages/statsmodels/stats/diagnostic.py:1081: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  aug = res.fittedvalues[:, None]


<Wald test (chi2): statistic=10778.507030495899, p-value=0.0, df_denom=2>


In [60]:
X3.isna().sum()

const              0
aso                0
capacity           0
new                0
first_owner        0
horse_power        0
mileage            0
no_accidents       0
number_of_doors    0
automatyczna       0
year               0
Audi               0
BMW                0
Bentley            0
Chevrolet          0
Citroën            0
Dacia              0
Ferrari            0
Fiat               0
Kia                0
Lamborghini        0
Land Rover         0
Lexus              0
MINI               0
Maserati           0
Mazda              0
McLaren            0
Mercedes-Benz      0
Mitsubishi         0
Porsche            0
Renault            0
Rolls-Royce        0
Saab               0
Seat               0
Suzuki             0
Toyota             0
Volvo              0
Škoda              0
Benzyna+LPG        0
Diesel             0
Hybryda            0
dtype: int64